In [ ]:
#| default_exp 09-faster-trie-implementation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import os, pandas as pd, warnings, torch, pickle, numpy as np
from typing import Dict, Optional, List
from tqdm.auto import tqdm
from scipy import stats
import scipy.sparse as sp
import torch.nn.functional as F
from itertools import chain

from xcai.basics import *
from xcai.models.MMM00X import DBT007, DBT008
from xcai.transform import AugmentMetaInputIdsTfm

In [ ]:
#| export
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
dump_dir = '/scratch/scai/phd/aiz218323/Projects/xc_nlg/outputs/05-metadata-augmented-input-and-trie-for-distilbert/'

In [ ]:
#| export
fname = '/scratch/scai/phd/aiz218323/Projects/xc_nlg/outputs/05-metadata-augmented-input-and-trie-for-distilbert/data/block.pkl'
with open(fname, 'rb') as f: block, test_dset = pickle.load(f)

## `Trie`

In [ ]:
class Trie:
    def __init__(self, max_height=32, sos_id=101, eos_id=102, pad_token=0, n_bm=10, len_penalty=1.0):
        store_attr('max_height,sos_id,eos_id,pad_token,n_bm,len_penalty')
        self.trie, self.hash = {}, None
    
    def build(self, X, y):
        assert(len(X) == len(y))
        self.hash = y
        trie_dict = {}
        for seq_id, seq in enumerate(tqdm(X)):
            next_dict = trie_dict
            for token in seq[:self.max_height]:
                next_dict[token] = next_dict.get(
                    token, {"next": {}, "occurs":0, "lbls": [],
                            "point_to": token, "is_leaf": False}
                )
                next_dict[token]["lbls"].append(seq_id)
                next_dict[token]["occurs"] += 1
                if token == self.eos_id:
                    next_dict[token]["is_leaf"] = True
                    break
                next_dict = next_dict[token]["next"]
        self.trie = trie_dict
    
    def decode_text(self, X):
        next_level = {'next':self.trie, 'lbls': [-1]}
        for token in X:
            items = next_level['next'].get(token, None)
            if items is None:
                return next_level["lbls"]
            if items["is_leaf"]:
                return items["lbls"]
            next_level = items
        return items["lbls"]
    
    def _padded_np(self, lol, fill_value, max_seq):
        tokens = np.full((len(lol), max_seq), fill_value, dtype=type(lol[0][0]))
        masks = np.zeros((len(lol), max_seq), dtype=np.int32)
        for i in np.arange(len(lol)):
            tokens[i, :len(lol[i])] = lol[i]
            masks[i, :len(lol[i])] = 1
        return tokens, masks
    
    def row_topk(self, score, k=10, return_scores=False, sort=False):
        index = np.argpartition(score, -k, axis=0)[-k:]
        if sort:
            _score = score[index]
            _index = np.argsort(_score, axis=0)
            index = index[_index]
        if return_scores:
            score = score[index]
            return index, score
        return index
    
    def batch_topk(self, score, k=10, return_scores=False, sort=False):
        index = np.argpartition(score, -k, axis=1)[:, -k:]
        if sort:
            _score = np.take_along_axis(score, index, axis=1)
            _index = np.argsort(_score, axis=1)
            index = np.take_along_axis(index, _index, axis=1)
        if return_scores:
            score = np.take_along_axis(score, index, axis=1)
            return index, score
        return index
    
    def _snl_one(self, tries, old_scores, curr_score, len_score, top_k_index):
        _tries, _score, _l_scr = [], [], []
        for col in top_k_index:
            items = list(tries[col]['next'].items())
            if len(items) == 0:
                key, value = [self.pad_token], [{"next":{}, "lbls": tries[col]["lbls"]}]
                l_items = [len_score[col]]
            else:
                key, value = list(zip(*items))
                l_items = np.full((len(key),), len_score[col] + 1)
            _tries.extend(value)
            _score.extend(curr_score[list(key)] + old_scores[col])
            _l_scr.extend(l_items)
        return _tries, _score, _l_scr
    
    def _agl_one(self, lol, sorted_index, sorted_scores):
        lol_lbs, lbl_scr = [], []
        for col, item in enumerate(sorted_index):
            _items = np.concatenate(
                        list(map(lambda x: self.hash[x], lol[item]["lbls"]))
                    )
            lol_lbs.extend(_items)
            lbl_scr.extend(np.full((_items.size,), sorted_scores[col], dtype=np.float32))
        return lol_lbs, lbl_scr

    def snl(self, tries, old_scores, curr_score, len_score, top_k_index):
        _tries, _score, _l_scr, _max_seq = [], [], [], -1
        for i in np.arange(top_k_index.shape[0]):
            __tries, __score, __l_scr = self._snl_one(
                tries[i], old_scores[i], curr_score[i], len_score[i], top_k_index[i])
            _max_seq = max(_max_seq, len(__tries))
            _tries.append(__tries)
            _score.append(__score)
            _l_scr.append(__l_scr)
        _score, _ = self._padded_np(_score, -np.inf, _max_seq)
        _l_scr, _ = self._padded_np(_l_scr, 0.001, _max_seq)
        return _tries, _score, _l_scr
    
    def agl(self, lol_trie, sorted_index, sorted_scores):
        lol_lbs, lbl_scr = [], []
        for rid in np.arange(sorted_index.shape[0]):
            _lol_lbs, _lbl_scr = self._agl_one(lol_trie[rid], sorted_index[rid], sorted_scores[rid])
            lol_lbs.append(_lol_lbs)
            lbl_scr.append(_lbl_scr)
        return lol_lbs, lbl_scr
    
    def decode_batch(self, preds, beam=10, l_penalty=0, start_seq=1):
        _token = np.full((len(preds), 1), self.sos_id)
        _tries = [[self.trie[self.sos_id]] for _ in range(len(preds))]
        _score = np.zeros((len(preds), 1), dtype=np.float32)
        _index = np.zeros((len(preds), 1), dtype=np.int32)
        _l_scr = np.ones((len(preds), 1), dtype=np.int32)
        for i in np.arange(start_seq, self.max_height):
            _tries, _score, _l_scr = self.snl(_tries, _score, preds[:, i], _l_scr, _index)
            _score = np.multiply(_score, np.power(_l_scr,-l_penalty))
            _index = self.batch_topk(_score, beam, i+1==self.max_height, i+1==self.max_height)
        return self.agl(_tries, _index[0], _index[1])

    def proc(self, model, inputs:Dict, n_bm:int=None, max_bm:Optional[int]=None, len_penalty:Optional[float]=None, 
             max_info:Optional[int]=None):
        store_attr('n_bm,len_penalty', is_none=False)
        logits = F.log_softmax(model(**inputs).logits, dim=2).cpu().detach().numpy()
        logits = np.concatenate([logits, np.zeros((logits.shape[0], max(0, self.max_height-logits.shape[1]), logits.shape[2]))], axis=1)
        idx, scores = self.decode_batch(logits, beam=self.n_bm, l_penalty=self.len_penalty)
        outputs = {
            'info2seq2data_idx': torch.tensor(list(chain(*idx))),
            'info2seq2data_score': torch.tensor(list(chain(*scores))),
            'info2seq2data_data2ptr': torch.tensor([len(o) for o in idx]),
        }
        return outputs
        
        
    def decode_one(self, pred, beam, l_penalty, start_seq=1):
        _token = [self.sos_id]
        _tries = [self.trie[self.sos_id]]
        _score = np.zeros((1,), dtype=np.float32)
        _index = np.zeros((1,), dtype=np.int32)
        _l_scr = np.ones((1, ), dtype=np.int32)
        for i in np.arange(start_seq, self.max_height):
            _tries, _score, _l_scr = self._snl_one(_tries, _score, pred[i], _l_scr, _index)
            _score = np.multiply(_score, np.power(_l_scr,-l_penalty))
            _index = self.row_topk(_score, beam, i+1==self.max_height, i+1==self.max_height)
        return self._agl_one(_tries, _index[0], _index[1])
    
    def decode_serial(self, preds, beam=10, l_penalty=0):
        labels = []
        scores = []
        for pred in preds:
            _labels, _scores = self.decode_one(pred, beam, l_penalty)
            labels.append(_labels)
            scores.append(_scores)
        return labels, scores

## Generation

In [ ]:
#| export
mname = f'/scratch/scai/phd/aiz218323/Projects/xc_nlg/outputs/05-metadata-augmented-input-and-trie-for-distilbert/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-WikiSeeAlsoTitles-320K/checkpoint-190000'
model = DBT007.from_pretrained(mname, tn_targ=10_000, ig_tok=0)

Some weights of DBT007 were not initialized from the model checkpoint at /scratch/scai/phd/aiz218323/Projects/xc_nlg/outputs/05-metadata-augmented-input-and-trie-for-distilbert/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-WikiSeeAlsoTitles-320K/checkpoint-190000 and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#| export
args = XCLearningArguments(
    output_dir=f'{dump_dir}/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-WikiSeeAlsoTitles-320K',
    generation_length_penalty=1.5,
    per_device_eval_batch_size=16,
    evaluation_strategy='steps',
    label_names=['lbl2data_idx'],
)

In [ ]:
#| export
metric = PrecRecl(test_dset.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[2, 3, 10, 50, 100, 200])

In [ ]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    data_collator=block.collator, 
    compute_metrics=metric,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trie = Trie(max_height=32, n_bm=10)
lbl_toks = test_dset.lbl_info['input_ids']
lbl_info = [[i] for i in range(len(toks))]
trie.build(lbl_toks, lbl_info)

  0%|          | 0/312330 [00:00<?, ?it/s]

In [ ]:
learn.tbs = trie

In [ ]:
o = learn.predict(test_dset)
display_metric(o.metrics)

/tmp/ipykernel_21032/2768843144.py:120: RuntimeWarning: divide by zero encountered in power
  _score = np.multiply(_score, np.power(_l_scr,-l_penalty))


/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@2,R@3,R@10,R@50,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,8.35,8.9,8.61,4.38,8.35,11.7853,14.8046,14.5668,6.3292,9.697,12.4711,11.4596,6.3292,8.9395,11.0119,10.9659,8.2428,12.6652,20.3485,20.3485,20.3485,20.3485,7.0555,209.895,9.529,0.3
